# Technical Analysis Functions and New DB

In [59]:
import sqlite3
import pandas as pd
import numpy as np
import datetime

In [80]:
# Get some stock prices from our Prices Database
# Load a list of ticker symbols to load
con1 = sqlite3.connect("TSX_Prices.sqlite")
#con2 = sqlite3.connect("TSX_Analysis.sqlite")
ticker = "SHOP"
sql = f"SELECT * FROM 'prices_daily' WHERE Ticker = '{ticker}' ORDER BY date ASC"
df = pd.read_sql_query(sql, con1)
df.drop(['index'], axis=1, inplace=True)
df["Date"] = pd.to_datetime(df["Date"], infer_datetime_format=True)
df.set_index("Date")
df.head()

,Date,Ticker,Open,High,Low,Close,Volume
0,2014-08-26,SHOP,-,-,-,0.04,0.0
1,2014-08-27,SHOP,-,-,-,0.04,0.0
2,2014-08-28,SHOP,-,-,-,0.04,0.0
3,2014-08-29,SHOP,-,-,-,0.04,0.0
4,2014-09-01,SHOP,-,-,-,0.04,0.0


In [137]:
df["previous_close"] = df["Close"].shift(1)
df["returns_manual"] = (df["Close"] / df["previous_close"]) - 1
df["change %"] = df["Close"] - df["previous_close"]
df["returns"] = df["Close"].pct_change()
# LN(1+return) -> EXP(sum of LN) - 1
df["log_return"] = np.log(1 + df["returns"])
df["cum_log"] = np.exp(df["log_return"].cumsum()) - 1
df.tail(10)

,Date,Ticker,Open,High,Low,Close,Volume,previous_close,returns_manual,change %,returns,log_return,cum_log
1797,2021-12-16,SHOP,1815.21,1815.21,1660.86,1693.69,307372.0,1753.59,-0.034158,-59.90,-0.034158,-0.034756,42341.25
1798,2021-12-17,SHOP,1672.0,1731.71,1645.0,1705.37,447966.0,1693.69,0.006896,11.68,0.006896,0.006873,42633.25
1799,2021-12-20,SHOP,1693.54,1710.63,1649.58,1662.21,144553.0,1705.37,-0.025308,-43.16,-0.025308,-0.025634,41554.25
1800,2021-12-21,SHOP,1703.04,1770.87,1687.41,1770.16,154336.0,1662.21,0.064944,107.95,0.064944,0.062922,44253.00
1801,2021-12-22,SHOP,1764.83,1786.79,1745.0,1776.67,144806.0,1770.16,0.003678,6.51,0.003678,0.003671,44415.75
1802,2021-12-23,SHOP,1788.0,1847.9,1772.87,1840.00,138957.0,1776.67,0.035645,63.33,0.035645,0.035025,45999.00
1803,2021-12-24,SHOP,1828.06,1849.99,1812.37,1813.55,21461.0,1840.00,-0.014375,-26.45,-0.014375,-0.014479,45337.75
1804,2021-12-29,SHOP,1770.18,1791.14,1730.13,1781.47,162865.0,1813.55,-0.017689,-32.08,-0.017689,-0.017847,44535.75
1805,2021-12-30,SHOP,1776.31,1833.45,1776.31,1781.51,156278.0,1781.47,0.000022,0.04,0.000022,0.000022,44536.75
1806,2021-12-31,SHOP,1781.39,1781.4,1740.77,1741.69,98397.0,1781.51,-0.022352,-39.82,-0.022352,-0.022605,43541.25


In [136]:
p1 = df.loc[df["Date"] == "2014-08-26"]["Close"].values[0]
p2 = df.loc[df["Date"] == "2021-12-31"]["Close"].values[0]
r1 = (p2 / p1) - 1
r1

43541.25